In [1]:
TOKEN = 'TOKEN'

In [ ]:
#discord integration
!pip install discord
import discord
intents = discord.Intents.default()
intents.message_content = True
client = discord.Client(intents = intents)

#bot model
import torch
!pip install transformers
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead, TrainingArguments, Trainer, DataCollatorWithPadding, AutoModelForCausalLM

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Th3BossC/DialoGPT-medium-AICLUB_NITC")
model = AutoModelForCausalLM.from_pretrained("Th3BossC/DialoGPT-medium-AICLUB_NITC")
tokenizer.pad_token = tokenizer.eos_token

from pathlib import Path
save_path = Path("models/")
save_path.mkdir(parents = True, exist_ok = True)
torch.save(model, save_path / 'DiscordBotDioloGPTmedium.pth')

In [15]:
class Bot():
  def __init__(self):
    self.step = 0
    self.chat_history_ids : torch.LongTensor = []

  def create_bot_ids(self, usr):
    new_user_input_ids = tokenizer.encode(usr + tokenizer.eos_token, return_tensors = "pt")
    if self.step == 0:
      return new_user_input_ids
    else:
      return torch.cat([self.chat_history_ids, new_user_input_ids], dim = -1)
    

  def reply(self, usr):
    if usr.lower() == "bot power off":
      return "-1"
    #new_user_input_ids = tokenizer.encode(usr + tokenizer.eos_token, return_tensors = "pt")
    # bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim = -1) if self.step > 0 else new_user_input_ids
    bot_input_ids = self.create_bot_ids(usr = usr)
    self.chat_history_ids = model.generate(
      bot_input_ids, 
      max_length = 500, 
      pad_token_id = tokenizer.eos_token_id, 
      no_repeat_ngram_size = 3, 
      do_sample = True, 
      top_k = 100, 
      top_p = 0.7, 
      temperature = 0.8
    )
    self.step += 1
    return tokenizer.decode(self.chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens = True)

ai_chatbot = Bot()

# step = 0
# def reply(usr):
#   if (usr.lower() == "bot power off"):
#     return "-1"
#   new_user_input_ids = tokenizer.encode(usr + tokenizer.eos_token, return_tensors = "pt")
#   bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim = -1) if step > 0 else new_user_input_ids
#   chat_history_ids = model.generate (
#       bot_input_ids,
#       max_length = 500,
#       pad_token_id = tokenizer.eos_token_id,
#       no_repeat_ngram_size = 3,
#       do_sample = True,
#       top_k = 100,
#       top_p = 0.7,
#       temperature = 0.8
#   )
#   step += 1
#   return tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens = True)

In [16]:
@client.event
async def on_ready():
    print(f"We have logged in as {client.user}")

@client.event
async def on_message(message):
    username = str(message.author).split('#')[0]
    user_message = str(message.content)
    channel = str(message.channel.name)
    print(f"{username}: {user_message} ({channel})")

    if message.author == client.user:
        return
    if message.content.startswith("shutdown"):
        await client.close()
    bot_answer = ai_chatbot.reply(usr = user_message)
    if message.channel.name == 'talk-with-the-bot':
        if bot_answer == "-1":
            return
        else:
            await message.channel.send(bot_answer)
            return



In [ ]:
!pip install nest_asyncio
import nest_asyncio

nest_asyncio.apply()
client.run(TOKEN)